In [20]:
from os import listdir
import jsonlines
import itertools
import nltk
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log

#nltk.download()
tknzr = nltk.TweetTokenizer()

In [80]:
path = listdir('c')
path = sorted(path)
path = ['AA', 'AD', 'AE']
print(path)

data = []
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AD', 'AE']


### Подготваливаем заголовки

In [81]:
sw_ru = nltk.corpus.stopwords.words('russian')
morph = pm.MorphAnalyzer()


stems = []
unsup_symb = set('.,\(\):«»?!')
for i in trange(len(data)):
    id = data[i]['id']
    tokens = tknzr.tokenize(text=data[i]['title'])
    tokens = [t.lower() for t in tokens if t not in unsup_symb]
    tokens = [morph.parse(t)[0].normal_form for t in tokens]
    tokens = [t for t in tokens if t not in sw_ru]
    data[i]['title'] = ' '.join(tokens)
    stems.extend(tokens)
    
stems = sorted(set(stems))
print("Done")
# Удалим статьи без названий
data = [d for d in data if d['title'] != '']

100%|██████████████████████████████████████████████████████████████████████████| 28267/28267 [00:17<00:00, 1597.40it/s]


Done


In [82]:
#titles_set = sorted(set(titles))
titles_set = {}
for i in trange(len(data)):
    if titles_set.get(data[i]['title']) is not None:
        titles_set[data[i]['title']].append(data[i]['id'])
    else:
        titles_set[data[i]['title']] = [data[i]['id']]

100%|████████████████████████████████████████████████████████████████████████| 28255/28255 [00:00<00:00, 181220.51it/s]


## Подготавливаем категории

In [83]:
categories = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories))))

categories_dict = {c: 0 for c in categories}
for d in tqdm(data):
    for c in d['categories']:
        if categories_dict.get(c) is not None:
            categories_dict[c] += 1
            
ok = [c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000]
len(ok)

100%|████████████████████████████████████████████████████████████████████████| 28255/28255 [00:00<00:00, 368405.20it/s]


3681

In [84]:
#categories_dict = {c: [d['id'] for d in data if c in d['categories']] for c in ok}
categories_dict = {}
for c in tqdm(ok):
    categories_dict[c] = [d['id'] for d in data if c in d['categories']]

print("Done")

100%|█████████████████████████████████████████████████████████████████████████████| 3681/3681 [00:34<00:00, 107.39it/s]


Done


## Вычисляем словарь категорий

In [85]:
categories_vocab = {}

for d in tqdm(data):
        
    for c in d['categories']:
        if c in ok:
            if categories_vocab.get(c) is None:
                categories_vocab[c] = d['title']
            else:
                categories_vocab[c] += ' ' + d['title']

for c in categories_vocab:
    categories_vocab[c] = set(categories_vocab[c].split())
                
    '''vocab = set()
    for id in c[1]:
        titles = [key for key, value in titles_set.items() if id in value]
        for t in titles:
            vocab.update(re.split(r' ', t))
    categories_vocab[c[0]] = vocab
    '''


100%|██████████████████████████████████████████████████████████████████████████| 28255/28255 [00:07<00:00, 3712.14it/s]


## Подготавливаем тексты _(как оказалось, это надо делать только для рассматриваемого документа)_

In [86]:
#texts = [d['text'] for d in data]
words = []

#CountVectorizer на таких объёмах не работал
with jsonlines.open('texts.jl', 'w') as output:
    for d in tqdm(data):

        text = re.sub(r'\n+', '. ', d['text'])
        text = re.sub(r'\xa0', ' ', text)
        tokens = tknzr.tokenize(text=text)
        tokens = [t.lower() for t in tokens]     #  if t.isalnum()
        text = ' '.join(tokens)
        output.write({'text': text, 'id': d['id']})
        words.extend(sorted(set([t for t in tokens if t.isalpha()])))

print("Done")
words = sorted(set(words))

100%|███████████████████████████████████████████████████████████████████████████| 28255/28255 [01:54<00:00, 245.90it/s]


Done


In [87]:
# Велосипедим
def binary_search(a, x, lo=0, hi=None):
    hi = hi if hi is not None else len(a)
    pos = bisect_left(a, x, lo, hi)
    return (pos if pos != hi and a[pos] == x else -1)


normal_forms = {}
for w in tqdm(words):
    nf = morph.parse(w)[0].normal_form
    if binary_search(stems, nf) != -1:
        normal_forms[w] = nf

100%|████████████████████████████████████████████████████████████████████████| 779432/779432 [03:43<00:00, 3479.71it/s]


In [88]:
with jsonlines.open('texts.jl', 'r') as file:
    with jsonlines.open('normalized_texts.jl', 'w') as output:
        for f in tqdm(file):
            id = d['id']
            text = f['text']
            tokens = re.split(r' ', text)
            tokens = [normal_forms.get(t) for t in tokens if normal_forms.get(t) is not None]
            text = ' '.join(tokens)
            output.write({'text': text, 'id': id})


28255it [00:19, 1451.36it/s]


## Начинаем анализировать документ

In [89]:
document = ''
i_d = 1
with jsonlines.open('normalized_texts.jl', 'r') as input:
    for i, f in enumerate(input):
        if i == i_d:
            document = f['text']
            break

data[1]

{'categories': ['Программное обеспечение'],
 'id': '6033',
 'text': 'Программное обеспечение\n\nПрогра́ммное обеспе́чение (допустимо также произношение "обеспече́ние") (ПО)\xa0— все или часть программ, процедур, правил и соответствующей документации системы обработки информации ("ISO/IEC 2382-1:1993").\n\nДругие определения из международных и российских стандартов:\n\nПрограммное обеспечение является одним из видов обеспечения вычислительной системы, наряду с техническим (аппаратным), математическим, информационным, лингвистическим, организационным и методическим обеспечением.\n\nПрограммное обеспечение\xa0— это то, что делает компьютеры универсальными, позволяя использовать типовую вычислительную машину для решения самых разнообразных задач.\n\nАкадемические области, изучающие программное обеспечение,\xa0— это информатика и программная инженерия.\n\nВ компьютерном сленге часто используется слово «"софт"», произошедшее от английского слова «"software"», которое в этом смысле впервые пр

1. Взвешиваем слова в документe по формуле $ R_w = ft_w \log{\frac{N}{cf_w}} $

In [90]:
words = re.split(r' ', document)
words_set = set(words)
# Вычисляем tf_w
R_w = dict(Counter(words))
N = len(categories_dict)
# Вычисляем log(...)
for w in tqdm(words_set):
    cf_w = 0
    for c in categories_vocab:
        if w in categories_vocab[c]:
            cf_w += 1
    if cf_w > 0:
        R_w[w] = R_w[w] * log(N / cf_w)
    else:
        R_w[w] = 0
        

100%|██████████████████████████████████████████████████████████████████████████████| 305/305 [00:00<00:00, 1445.08it/s]


In [91]:
sorted([k for k in R_w.items() if k[1] > 0], key=lambda t: t[1], reverse=True)

[('программный', 291.60542523314734),
 ('обеспечение', 256.0437880095928),
 ('программа', 102.36968058388696),
 ('компьютер', 83.44934343396255),
 ('год', 71.0686774140787),
 ('который', 63.8954402219639),
 ('продукт', 60.14234042255261),
 ('система', 56.573186062950484),
 ('машина', 45.46742518420632),
 ('вычислительный', 45.10675531691446),
 ('рынок', 45.10675531691446),
 ('пользователь', 45.10675531691446),
 ('разработка', 44.93426184905676),
 ('фирма', 37.59017750594225),
 ('бизнес', 37.59017750594225),
 ('персональный', 37.58896276409538),
 ('статья', 34.12322686129565),
 ('компьютерный', 33.007509104724605),
 ('процессор', 32.843758933516085),
 ('появиться', 32.843758933516085),
 ('это', 32.39194556446038),
 ('код', 32.09590132075483),
 ('тьюринг', 31.325147921618537),
 ('программирование', 29.06522230290325),
 ('база', 28.449309778843645),
 ('задача', 27.298581489036522),
 ('компания', 26.888631946614026),
 ('доллар', 26.888631946614026),
 ('управление', 25.676721056603864),
 ('

2\. Взвешиваем заголовки по формуле $R_t = \sum_{w \rightarrow t} R_w \frac{1}{t_w} \frac{1}{a_t} \frac{S_t}{L_t}$

In [92]:
R_t = dict()
title_vocab = dict(Counter(' '.join(titles_set.keys()).split()))
for title, ids in titles_set.items():
    title_words = title.split()
    supp_words = []
    flag = 1
    for w in title_words:
        if w not in words_set:
            flag = flag - 1
        else:
            supp_words.append(w)
    if flag < 0:
        continue
    sub_sum = 0
    # Вынесем 1 / a_t * S_t / L_t как mutual_mul
    mutual_mul = len(supp_words) / (len(titles_set[title]) * len(title_words))
    for w in supp_words:
        sub_sum += R_w[w] / title_vocab[w]
        
    R_t[title] = mutual_mul * sub_sum

In [93]:
sorted([k for k in R_t.items() if k[1] > 0], key=lambda t: t[1], reverse=True)

[('прикладной программный обеспечение', 39.71049489963592),
 ('системный программный обеспечение', 39.54856829949888),
 ('свободный программный обеспечение', 35.956370285165505),
 ('бесплатный программный обеспечение', 35.44309843280937),
 ('программный обеспечение', 35.44309843280937),
 ('руководство пользователь', 30.76431739183625),
 ('интерфейс пользователь', 24.195565605133034),
 ('инструментальный программный обеспечение', 23.62873228853958),
 ('сервер программный обеспечение', 23.62873228853958),
 ('взлом программный обеспечение', 23.62873228853958),
 ('проприетарный программный обеспечение', 23.62873228853958),
 ('условно-бесплатный программный обеспечение', 23.62873228853958),
 ('открытый программный обеспечение', 23.62873228853958),
 ('портирование программный обеспечение', 23.62873228853958),
 ('acme программный обеспечение', 23.62873228853958),
 ('графический процессор', 16.421879466758043),
 ('конкретный', 15.035585105638152),
 ('идея', 14.224654889421823),
 ('персональный

3\. Возвешиваем статьи

In [94]:
R_a = {d['id']: R_t.get(d['title']) for d in data if R_t.get(d['title']) is not None and R_t.get(d['title']) > 0.0}

4\. Взвештваем категории и получам ответ

In [95]:
R_c = {}
for c in ok:
    r_c = 0
    for id in categories_dict[c]:
        if R_a.get(id) is not None:
            r_c += R_a.get(id)
    R_c[c] = r_c
    
sorted([k for k in R_c.items() if k[1] > 0], key=lambda t: t[1], reverse=True)

[('Программное обеспечение', 134.936217740824),
 ('Кинокомедии США', 46.99146083375654),
 ('Фильмы на английском языке', 46.66553979226978),
 ('Техническая документация', 39.037307622790735),
 ('Программирование', 37.65124496649075),
 ('История компьютерной техники', 27.34773165130291),
 ('Интерфейс пользователя', 24.195565605133034),
 ('Shareware', 23.655469264678374),
 ('Теория алгоритмов', 23.395274913787745),
 ('Философская логика', 21.829871668790183),
 ('Романтические кинокомедии США', 20.375654259562786),
 ('Фильмы-триллеры США', 18.87308941073647),
 ('Фильмы-драмы Франции', 18.22031987880363),
 ('Психологические понятия', 17.86781043314951),
 ('Компьютеры СССР', 17.071016891011652),
 ('Понятия метафизики', 16.36438164413881),
 ('Эпистемология', 15.404857270662212),
 ('Фильмы-драмы США', 15.3696345276104),
 ('Греческие философские термины', 14.224654889421823),
 ('Фильмы-мелодрамы США', 13.943497218721976),
 ('Офисная техника', 13.789311398074506),
 ('Проектирование', 13.6777539

## Ниже черновик

In [ ]:
titles_set = sorted(set(titles))
print(len(titles))
print(titles_set[0:100])


739695
['', ' аистёс', '0', '0 2', '0 9', '0 a d', '0 е год до наш э', '0 сапфо', '0 январь', '00 династия', '00 крессмания', '001', '007 legends', '007 координата скайфолл', '007 спектр', '01 гельверция', '010 альбом the mad capsule markets', '01011001', '013 гордонмур', '034 акка', '04 испания', '05 гормуция', '05а 192 автодорога', '05а 214 автодорога', '05а 215 автодорога', '05а 608 автодорога', '05к 195 автодорога', '05к 442 автодорога', '05к 451 автодорога', '05н 100 автодорога', '05н 131 автодорога', '05н 263 автодорога', '060 аний', '07 вестур', '08а 10 автодорога', '09 лундия', '0verflow', '1', '1 1 1', '1 1 international', '1 1 медиа', '1 1 телеканал', '1 1 фильм', '1 2', '1 4', '1 9 8 2', '1 fille 4 types', '1 firebird', '1 gapa', '1 the collection', '1 wire', '1 август', '1 август пилотажный группа', '1 альбом хулио иглесиас', '1 апрель', '1 бром 3 хлор диметилгидантоин', '1 го отделение совхоз масловский', '1 го отделение совхоз новоусманский', '1 го ранг', '1 декабрь', '1 

In [7]:
title_to_article = nx.Graph()
for i, t in enumerate(titles):
    if i % 1111 == 0:
        print(i)
    title = next((ttl for ttl in titles_set if ttl == t), None)
    title_to_article.add_edge(title, int(data[i]['id']))


0
1111
2222
3333
4444
5555
6666
7777
8888
9999
11110
12221
13332
14443
15554
16665
17776
18887
19998
21109
22220


KeyboardInterrupt: 

In [ ]:
def binary_search(a, x, lo=0, hi=None):
    hi = hi if hi is not None else len(a)
    pos = bisect_left(a, x, lo, hi)
    return (pos if pos != hi and a[pos] == x else -1)



def process_article(i, t):
    if i % 101 == 0:
        print(i)
        
    tokens = tknzr.tokenize(text = t)
    tokens = [t.lower() for t in tokens if t.isalnum()]
    
    #вот тут скорее всего будет оч долго работать
    t = ''
    for tok in tokens:
        nf = morph.parse(tok)[0].normal_form
        if binary_search(stems, nf) != -1:
            t = t + nf + ' '
            
    t = t[:-1]

    
texts = Parallel(n_jobs=2)(delayed(process_article)(i, t) for i, t in enumerate(texts))

In [17]:
'''
categories = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories))))
print("Categories - " + str(len(categories)))
print("Articles - " + str(len(data)))

categories
'''

'\ncategories = [d[\'categories\'] for d in data]\ncategories = sorted(set(list(itertools.chain(*categories))))\nprint("Categories - " + str(len(categories)))\nprint("Articles - " + str(len(data)))\n\ncategories\n'

In [18]:
'''
vectorizer = CountVectorizer()
vectorizer.fit_transform([texts[0]])
vectorizer.vocabulary_
'''

'\nvectorizer = CountVectorizer()\nvectorizer.fit_transform([texts[0]])\nvectorizer.vocabulary_\n'

In [53]:
nltk.word_tokenize('Я человек.')

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/maxfelmos/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [3]:
path = listdir('c')
i = 0
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            i += 1

print(i)

739450


{'!', '(', ')', ',', '.', ':', '?', '«', '»'}